<a href="https://colab.research.google.com/github/pietrzakkuba/computer-assisted-translation-labs/blob/main/lab_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Komputerowe wspomaganie tłumaczenia

# Zajęcia 7 - web scraping

Jak dobrze wiemy, w procesie wspomagania tłumaczenia oraz w zagadnieniach przetwarzania języka naturalnego ogromną rolę pełnią zasoby lingwistyczne. Należą do nich korpusy równoległe (pamięci tłumaczeń), korpusy jednojęzyczne oraz słowniki. Bywa, że zasoby te nie są dostępne dla języka, nad którym chcemy pracować.

W tej sytuacji jest jeszcze dla nas ratunek - możemy skorzystać z zasobów dostępnych publicznie w Internecie. Na dzisiejszych zajęciach przećwiczymy techniki pobierania tekstu ze stron internetowych.

Poniższy kod służy do ściągnięcia zawartości strony (w formacie HTML do zmiennej) oraz do wyszukania na tej stronie konkretnych elementów. Przed jego uruchomieniem należy zainstalować moduł BeautifulSoup:
`pip3 install beautifulsoup4`

In [1]:
import requests
from bs4 import BeautifulSoup

url='https://epoznan.pl'

page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

headers = soup.find_all('h3', {'class':'postItem__title'})

print('\n'.join([header.get_text() for header in headers]))

Za moment rozkwitną magnolie. W kórnickim arboretum robi się coraz piękniej
Podszywają się pod rachmistrzów i próbują wchodzić do mieszkań
Śmiertelny wypadek na trasie Rawicz - Krotoszyn. Ciężarówka zderzyła się z motocyklem
Miliony złotych strat, 152 oszukane osoby. Zatrzymano doradców finansowych. Także w Poznaniu
2 lata więzienia grożą mężczyźnie, który w sklepie uderzył kobietę. Usłyszał zarzuty
1429 nowych zakażeń w Wielkopolsce. Tylko w regionie zmarły 93 osoby
Zagraj z nami o swoją przyszłość i wygraj!
W wyniku braku tlenu na MTP zmarło 8 pacjentów?
Maciej Skorża: było widać radość z gry, zawodnicy tego potrzebują
Uzbierano 1,3 mln zł na operację małego Natana z Zaniemyśla. Potrzebne dodatkowe pieniądze
Zderzenie dwóch aut na Al. Niepodległości
Wyprzedaże w sklepach Tesco: sieć wycofuje się z Polski
Piotr Tworek: przegraliśmy dziwne spotkanie
Trasa na Naramowice: torowisko się zazieleniło!
Skorża ma pierwszą wygraną. Lech Poznań - Lechia Gdańsk 3:0
Zaplanowane na środę wyłączeni

### Ćwiczenie 1: Napisz funkcję do pobierania nazw towarów z serwisu Ceneo.pl. Typ towaru, np. telewizor, pralka, laptop jest parametrem funkcji. Wystarczy pobierać dane z pierwszej strony wyników wyszukiwania.

In [39]:
def get_names(article_type):
    url = 'https://www.ceneo.pl/' + article_type
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    tags = soup.find_all('a', {'class': 'go-to-product js_conv js_clickHash js_seoUrl'})
    for tag in tags:
        new_label = tag.find('span', {'class': 'new-label'})
        if new_label is not None:
            new_label.extract()
    return [tag.text.strip() for tag in tags]

get_names('Laptopy')

['Huawei MateBook D15 15,6"/Ryzen5/8GB/256GB/Win10 (53010TUE)',
 'Huawei Matebook D16 16,1"/Ryzen5/16GB/512GB/W10 (Harvey-WAP9D)',
 'Huawei MateBook D14 2020 14"/Ryzen7/8GB/512GB/Win10 (53010XTR)',
 'Apple MacBook Air 13,3"/M1/8GB/256GB/macOS (MGN63ZEA)',
 'Apple MacBook Air 13,3"/i5/8GB/128GB/macOS Srebrny (MQD32ZEA)',
 'ASUS X509JA-BQ241 15,6"/i5/8GB/512GB/Win10 (X509JABQ241T)',
 'Apple MacBook Air 13,3"/M1/8GB/256GB/macOS (MGND3ZEA)',
 'Huawei MateBook 14 2020 14"/Ryzen5/16GB/512GB/Win10 (53011JKH)',
 'Apple MacBook Pro 13,3"/M1/8GB/256GB/macOS (MYD82ZEA)',
 'HP Pavilion Gaming 16,1"/i5/16GB/512GB/Win10 (2C5W3EA)',
 'ASUS X515MA-BR210 15,6"/N4020/8GB/256GB/NoOS (X515MABR210)',
 'Acer Aspire 5 15,6"/R5/8GB/512GB/Win10 (A51544NXHW4EP005)',
 'Apple MacBook Air 13,3"/M1/16GB/256GB/macOS (MGN63ZEAR1)',
 'Lenovo Legion 5-15 15,6"/Ryzen5/8GB/512GB/NoOS (82B1007VPB)',
 'Acer Nitro 5 15,6"/Ryzen5/8GB/512GB/NoOS (NHQ6ZEP006)',
 'Dell Inspirion 3593 15,6"/i5/12GB/1TB/Win10 (NDI393E01E1HDFXXBKT

W ten sposób pobieramy dane z jednej strony. Nic jednak nie stoi nam na przeszkodzie, aby zasymulować przełączanie stron.

### Ćwiczenie 2: Zaobserwuj, jak zmienia się url strony podczas przechodzenia do kolejnych stron wyników wyszukiwania na Ceneo.pl. Wykorzystaj tę informację i uruchom funkcję get_names() na więcej niż jednej stronie wyników.

In [38]:
def get_names_pages(article_type, url_suffix):
    url = 'https://www.ceneo.pl/' + article_type + url_suffix
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    tags = soup.find_all('a', {'class': 'go-to-product js_conv js_clickHash js_seoUrl'})
    for tag in tags:
        new_label = tag.find('span', {'class': 'new-label'})
        if new_label is not None:
            new_label.extract()
    return [tag.text.strip() for tag in tags]

def scrape_names(article_type):
    names = []
    for i in range(10):
        names += get_names_pages(article_type, f';0020-30-0-0-{i}.htm')
    return names

scrape_names('Laptopy')

['Huawei MateBook D15 15,6"/Ryzen5/8GB/256GB/Win10 (53010TUE)',
 'Huawei Matebook D16 16,1"/Ryzen5/16GB/512GB/W10 (Harvey-WAP9D)',
 'Huawei MateBook D14 2020 14"/Ryzen7/8GB/512GB/Win10 (53010XTR)',
 'Apple MacBook Air 13,3"/M1/8GB/256GB/macOS (MGN63ZEA)',
 'Apple MacBook Air 13,3"/i5/8GB/128GB/macOS Srebrny (MQD32ZEA)',
 'ASUS X509JA-BQ241 15,6"/i5/8GB/512GB/Win10 (X509JABQ241T)',
 'Apple MacBook Air 13,3"/M1/8GB/256GB/macOS (MGND3ZEA)',
 'Huawei MateBook 14 2020 14"/Ryzen5/16GB/512GB/Win10 (53011JKH)',
 'Apple MacBook Pro 13,3"/M1/8GB/256GB/macOS (MYD82ZEA)',
 'HP Pavilion Gaming 16,1"/i5/16GB/512GB/Win10 (2C5W3EA)',
 'ASUS X515MA-BR210 15,6"/N4020/8GB/256GB/NoOS (X515MABR210)',
 'Acer Aspire 5 15,6"/R5/8GB/512GB/Win10 (A51544NXHW4EP005)',
 'Apple MacBook Air 13,3"/M1/16GB/256GB/macOS (MGN63ZEAR1)',
 'Lenovo Legion 5-15 15,6"/Ryzen5/8GB/512GB/NoOS (82B1007VPB)',
 'Acer Nitro 5 15,6"/Ryzen5/8GB/512GB/NoOS (NHQ6ZEP006)',
 'Dell Inspirion 3593 15,6"/i5/12GB/1TB/Win10 (NDI393E01E1HDFXXBKT

Technika pobierania treści z Internetu jest szczególnie efektywnym sposobem na pozyskiwanie dużych ilości tekstu. Poniższy fragment kodu służy do ściągnięcia całości tekstu ze strony.

In [72]:
import re

url = "https://www.yahoo.com"

page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

# usunięcie elementów script i style
for script in soup(["script", "style"]):
    script.extract()    # usuń element

# pobierz tekst
text = soup.get_text()

# usuń wielokrotne białe znaki
text = re.sub(r"\s+", " ", text)

print(text)

 Yahoo Make Yahoo Your HomepageDiscover something new every day from News, Sports, Finance, Entertainment and more! HOME MAIL NEWS FINANCE SPORTS ENTERTAINMENT LIFE SHOPPING YAHOO PLUS MORE... Download the Yahoo Home app Yahoo Home Search query Sign in Mail Sign in to view your mail Mail Mail COVID-19 COVID-19 News News Finance Finance Sports Sports Entertainment Entertainment Life Life Shopping Shopping Yahoo Plus Yahoo Plus More... More... 16-year-old Black girl fatally shot by police in OhioPolice responding to a report of an attempted stabbing fatally shot a teenage girl just before the guilty verdict was announced against the officer charged in George Floyd's death. Police release bodycam footage »The largest mass protest movement in U.S. history'Nope. Done.': Tucker Carlson abruptly ends interviewJane Fonda: 'My knees gave way' after actor's kiss'Today' host back under the knife for last time Pelosi under fire for response to Chauvin conviction CelebrityE!Ellen DeGeneres Admits H

### Ćwiczenie 3: Napisz program do pobrania tekstu ze strony Wydziału Matematyki i Informatyki. Pobierz cały tekst ze strony głównej a następnie wyszukaj na tej stronie wszystkich linków wewnętrznych i pobierz tekst ze stron wskazywanych przez te linki. Nie zagłębiaj się już dalej.


In [126]:
def scrape_by_url(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    for script in soup.find_all(['script', 'style']):
        script.extract()
    text = soup.text.strip()
    return re.sub(r"\s+", " ", text)

def scrape_subpages(url):
    text = ''
    urls = [url]
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    for a_tag in soup.find_all('a', href=True):
        link = a_tag['href']
        if link.startswith('/'):
            urls.append(url + link)
    for url in urls:
        text += scrape_by_url(url)
    return text


scrape_subpages('https://old.wmi.amu.edu.pl') # wersja old działa lepiej niż obecna

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

'Aktualności Użytkownik Hasło Zapamiętaj Zaloguj Nie pamiętasz nazwy? Nie pamiętasz hasła? Login Wersja dla niedowidzących Wersja zwyczajna Uniwersytet im. Adama Mickiewicza w Poznaniu Wydział Matematyki i Informatyki WydziałNauczanie zdalneNaukaDla studentaDla doktorantaDla pracownikaDla kandydataDla szkół Wydział AktualnościWybory 2020HistoriaRozkład roku akademickiegoStruktura wydziałuWładzePełnomocnicyKierownicy kierunkówPracownicyBibliotekaZarządzenia dziekanaNauczanie zdalneInformator wydziałowyRada Naukowa DyscyplinRada WydziałuJakość kształceniaAbsolwenci wydziałuKontaktZauważyłeś problem na stronie? Zgłoś! Szybkie linki Strona główna UAMEduroamHelpdeskSukcesy pracownikówSukcesy studentówSukcesy Wydziału XXVII Uroczysty Wykład im. Wojtka Pulikowskiego Zapraszamy na XXVII Uroczysty Wykład im. Wojtka Pulikowskiego, który odbędzie się1 grudnia 2020 r. o godzinie 10:00 zdalnie (link do spotkania). Wykład "Grafy bez długich indukowanych ścieżek" wygłosi dr hab. Marcin Pilipczuk z Wy

Omówione wyżej techniki działają również bardzo dobrze dla zasobów słownikowych.

### Ćwiczenie 4: Pobierz jak najwięcej słów w języku albańskim z serwisu glosbe.com.

In [186]:
import requests
from bs4 import BeautifulSoup

global_url = 'https://glosbe.com'

def scrape_shqip(url):
    # bez tych headersów rzucało 429 Too Many Requests :)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    page = requests.get(global_url + url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    word_list = soup.find('ul', {'id': 'sidebar-wordlist'})
    after_active = False
    children = word_list.findChildren('li' , recursive=False)
    children_len = len(children)
    for index, child in enumerate(children):
         # pobieramy tylko słowa poniżej obecnie wybranego, by nie powtarzać ich
        if child.has_attr('class') and 'active' in child['class'] and not after_active:
            after_active = True
        if after_active and not index == children_len - 1:
            print(child.text.strip())
        if index == children_len - 1:
            try:
                scrape_shqip(child.find('a')['href'])
            except:
                return

scrape_shqip('/sq/en/a')

a
A
â
A e ke problem?
a flet anglisht
a flet njeri anglisht?
a flisni anglisht
a flisni Shqip
a je fetar
a je fetar?
a je fetare
a je fetare?
a jeni alergjik ndaj ndonjë medikamenti
a jeni alergjik ndaj ndonjë medikamenti?
a jeni fetar
a jeni fetar?
a jeni fetare
a jeni fetare?
a keni nevojë për ndihmë?
A më kuptoni?
a predominant trait
Ab actis
abaci
abak
abakus
Abakusi
abandonoj
abanoz
Abanoz
abat
abate
abati
Abati
abazia
Abazia
abazhur
abazhuri
Abazhuri
abc
Abdesi
abdikim
abdikoj
abdomen
abdomeni
Abdomeni
Abdullah
abece
aberdeen
Aberdeen
abetare
Abetarja
abëcë
abidjan
abies
Abisinia
Abkhazia
ablacioni
Ablacioni
Ablaktacion
abolicionist
abolla
abonent
abonim
Abonimi
abonohem
abonoj
abonsina
abonuar
aborotoni
abort
aborti
Aborti
abrahami
Abrahami
abrakadabra
abrazioni
Abrazioni
abraziv
abrazivët
abrogim
abrogoj
Abruci
ABS
Abscesi i mëlçisë
absenteizëm
absolut
absolutisht
absolutizëm
absolutizmi
Absolutizmi
absorbeer
absorbim
absorboj
abstenim
abstenoj
abstenonjës
abstrakt
absurd
absur